In [3]:
import btrack
import zarr
import os
import napari
from macrohet import dataio
import glob
import numpy as np

In [5]:
expt_ID = 'ND0004'
base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,replicate_number=False)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
acq_ID = (3, 4)
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
zarr_group = zarr.open(image_dir, mode='r')
images = zarr_group.images[0,...]

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


In [6]:
images.shape

(2, 3, 6048, 6048)

In [7]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/cpv3/{acq_ID}.h5'), #macrohet_seg_model 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation[0]

[INFO][2024/10/22 03:29:52 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(3, 4).h5...
[INFO][2024/10/22 03:30:11 PM] Loading segmentation (154, 6048, 6048)
[INFO][2024/10/22 03:30:11 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(3, 4).h5


In [8]:
segmentation.shape

(6048, 6048)

In [16]:
viewer = napari.Viewer(title = 'testing nd4 segmentation and tracking')

viewer.add_image(images, channel_axis = 0, 
                 colormap=['magenta', 'green'],
                 blending = 'additive', 
                 contrast_limits=[[0, 1000], [0, 2400]])
viewer.add_labels(segmentation)

<Labels layer 'segmentation' at 0x7f93e59e09a0>

# Testing new approach

Segment the top z slice of the green channel... can always expand the masks later on to try and capture the mtb

In [17]:
zarr_group.images.shape

(154, 2, 3, 6048, 6048)

In [23]:
%%time
segmentation_input = zarr_group.images[:,1,2,...]

CPU times: user 9.72 s, sys: 5.24 s, total: 15 s
Wall time: 54.8 s


In [24]:
viewer.add_image(segmentation_input)

<Image layer 'segmentation_input' at 0x7f93e45de170>

In [27]:
from stardist.models import StarDist2D

# prints a list of available models
StarDist2D.from_pretrained()

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

2024-10-22 15:46:41.030031: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 15:46:41.060827: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [28]:
from csbdeep.utils import normalize



In [31]:
help(model.predict_instances)

Help on method _predict_instances_generator in module stardist.models.base:

_predict_instances_generator(img, axes=None, normalizer=None, sparse=True, prob_thresh=None, nms_thresh=None, scale=None, n_tiles=None, show_tile_progress=True, verbose=False, return_labels=True, predict_kwargs=None, nms_kwargs=None, overlap_label=None, return_predict=False) method of stardist.models.model2d.StarDist2D instance
    Predict instance segmentation from input image.
    
    Parameters
    ----------
    img : :class:`numpy.ndarray`
        Input image
    axes : str or None
        Axes of the input ``img``.
        ``None`` denotes that axes of img are the same as denoted in the config.
    normalizer : :class:`csbdeep.data.Normalizer` or None
        (Optional) normalization of input image before prediction.
        Note that the default (``None``) assumes ``img`` to be already normalized.
    sparse: bool
        If true, aggregate probabilities/distances sparsely during tiled
        predicti

In [43]:
%%time
labels, _ = model.predict_instances(normalize(segmentation_input[0]), scale = 0.5, prob_thresh = 0.3
                        )

viewer.add_labels(labels)

CPU times: user 8min 45s, sys: 695 ms, total: 8min 46s
Wall time: 22.2 s


<Labels layer 'labels [4]' at 0x7f9325773970>

In [37]:
viewer = napari.Viewer(title = 'testing nd4 seg')

In [38]:
viewer.add_labels(labels)

<Labels layer 'labels' at 0x7f93258eb400>

In [39]:
viewer.add_image(normalize(segmentation_input[0]))

<Image layer 'Image' at 0x7f938c33b190>

### Do not think Stardist is best suited to this approach

# Trying Cellpose

In [44]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics
use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')
model = models.Cellpose(gpu=True, model_type='cyto')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0
Tue Oct 22 16:02:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:65:00.0  On |                  Off |
| 30%   42C    P8      

In [46]:
%%time
masks, flows, styles, diams = model.eval(normalize(segmentation_input[0]), diameter=130, channels=[0,0],
                                             # flow_threshold=flow_threshold, cellprob_threshold=0
                                          )   
viewer.add_labels(masks)

CPU times: user 24.5 s, sys: 12.1 s, total: 36.5 s
Wall time: 34.8 s


<Labels layer 'masks' at 0x7f957b21b8b0>

In [53]:
%%time
masks, flows, styles, diams = model.eval(segmentation_input[0], 
                                         diameter=150, 
                                         channels=[0,0],
                                         # flow_threshold=0.2
                                             # flow_threshold=flow_threshold, cellprob_threshold=0
                                          )   
viewer.add_labels(masks)

CPU times: user 24.4 s, sys: 14.3 s, total: 38.7 s
Wall time: 36.9 s


<Labels layer 'masks [4]' at 0x7f957b2ddba0>

In [47]:
viewer.add_labels(segmentation)

<Labels layer 'segmentation' at 0x7f956a4a2620>

In [56]:
viewer.add_image(zarr_group.images[0,1,1
                ,...])

<Image layer 'Image [3]' at 0x7f957b685c90>

In [58]:
from tqdm.auto import tqdm

In [59]:
mask_stack = []
for frame in tqdm(segmentation_input, total = len(segmentation_input)):
    masks, flows, styles, diams = model.eval(frame, 
                                         diameter=150, 
                                         channels=[0,0],)

    mask_stack.append(masks)



  0%|          | 0/154 [00:00<?, ?it/s]

In [61]:
cellpose_segmentation = np.stack(mask_stack, axis = 0)

In [65]:
# Save the NumPy array to a file
np.save('/mnt/SYNO/macrohet_syno/temp_cellpose_masks.npy', cellpose_segmentation)

In [66]:
viewer.add_labels(cellpose_segmentation)

<Labels layer 'cellpose_segmentation' at 0x7f9583b2f0a0>

In [67]:
viewer.add_image(segmentation_input)

<Image layer 'segmentation_input' at 0x7f957b65df90>

# Trackastra

In [68]:
import torch
from trackastra.model import Trackastra
from trackastra.tracking import graph_to_ctc, graph_to_napari_tracks

In [84]:
%%time 

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load a pretrained model
model = Trackastra.from_pretrained("general_2d", device=device)

# or from a local folder
# model = Trackastra.from_folder('path/my_model_folder/', device=device)

# Track the cells
track_graph = model.track(segmentation_input, cellpose_segmentation, mode="ilp")  # or mode="ilp", or "greedy_nodiv"


# Write to cell tracking challenge format
ctc_tracks, masks_tracked = graph_to_ctc(
      track_graph,
      masks,
      outdir="tracked",
)

# Visualise in napari
napari_tracks, napari_tracks_graph, _ = graph_to_napari_tracks(track_graph)



INFO:trackastra.model.model:Loading model state from /home/dayn/.trackastra/.models/general_2d/model.pt
INFO:trackastra.model.model_api:Using device cuda
INFO:trackastra.model.model_api:Predicting weights for candidate graph


/home/dayn/.trackastra/.models/general_2d already downloaded, skipping.


INFO:trackastra.data.wrfeat:Extracting features from 154 detections
INFO:trackastra.data.wrfeat:Using single process for feature extraction
Extracting features: 100%|███████████████████████████████████████████████████████████| 154/154 [05:15<00:00,  2.05s/it]
INFO:trackastra.model.model_api:Building windows
Building windows: 100%|████████████████████████████████████████████████████████████| 151/151 [00:00<00:00, 4534.74it/s]
INFO:trackastra.model.model_api:Predicting windows
Computing associations: 100%|████████████████████████████████████████████████████████| 151/151 [00:45<00:00,  3.33it/s]
INFO:trackastra.model.model_api:Running greedy tracker
INFO:trackastra.tracking.tracking:Build candidate graph with delta_t=1
INFO:trackastra.tracking.tracking:Added 130836 vertices, 139397 edges                                                  
INFO:trackastra.tracking.ilp:Using `gt` ILP config.


AttributeError: 'Solver' object has no attribute 'add_costs'

In [71]:
cellpose_segmentation.shape

(154, 6048, 6048)

INFO:trackastra.model.model:Loading model state from /home/dayn/.trackastra/.models/general_2d/model.pt
INFO:trackastra.model.model_api:Using device cuda


/home/dayn/.trackastra/.models/general_2d already downloaded, skipping.


INFO:trackastra.model.model_api:Predicting weights for candidate graph


Tracking with mode greedy_nodiv...


INFO:trackastra.data.wrfeat:Extracting features from 154 detections
INFO:trackastra.data.wrfeat:Using single process for feature extraction
INFO:trackastra.model.model_api:Building windows
INFO:trackastra.model.model_api:Predicting windows
INFO:trackastra.model.model_api:Running greedy tracker
INFO:trackastra.tracking.tracking:Build candidate graph with delta_t=1
INFO:trackastra.tracking.tracking:Added 130836 vertices, 132759 edges                                                  
INFO:trackastra.tracking.tracking:Running greedy tracker
Converting graph to CTC results: 100%|████████████████████████████████████████████| 6979/6979 [00:13<00:00, 507.00it/s]
INFO:napari_ctc_io._writer:Writing CTC format to /home/dayn/analysis/macrohet                                          
INFO:napari_ctc_io._writer:Writing CTC format to /home/dayn/analysis/macrohet


In [ ]:
import napari
v = napari.Viewer()
v.add_image(imgs)
v.add_labels(masks_tracked)
v.add_tracks(data=napari_tracks, graph=napari_tracks_graph)

In [74]:
tracks = viewer.layers['tracks'].data

In [78]:
tracks

array([[1.00000000e+00, 0.00000000e+00, 5.63499262e+03, 5.98160372e+03],
       [1.00000000e+00, 1.00000000e+00, 5.62536602e+03, 5.96917297e+03],
       [1.00000000e+00, 2.00000000e+00, 5.61263586e+03, 5.95262619e+03],
       ...,
       [6.97800000e+03, 1.53000000e+02, 5.03416957e+03, 5.25425019e+03],
       [6.97900000e+03, 1.52000000e+02, 5.29644164e+03, 6.23781633e+02],
       [6.97900000e+03, 1.53000000e+02, 5.32309206e+03, 6.78151989e+02]])

In [80]:
import pandas as pd

In [82]:
ztracks_df = pd.DataFrame(tracks, columns=['ID', 'T', 'X', 'Y'])
ztracks_df.to_csv('/mnt/SYNO/macrohet_syno/data/ND0004/labels/testing_trackastra/ztracks.csv')